## Table of Contents
- [Introduction](#introduction)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Import](#import)
    - [Assess](#assess)
    - [Clean](#clean)
    - [Store](#store)

## I) Introduction <a id = "introduction">

**Broad question:** How do price forecasts for each firm in the S&P 2019 Index compare to their corresponding actual prices?

**Approach:** Analyze difference in means between average forecast EPS and average actual EPS for each firm.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.

[TK] HEre is a breakdown of my final clean CSV's features, TK.csv.

## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual price earning dates by *calendar period* instead of fiscal period. This is because fiscal period differs by firm whereas calendar period is consistent by dates. 

#### Through the Bloomberg Excel functions, I gathered four datasets with different purposes:

- historical forecasted EPS
- historical actual EPS
- historical actual EOD price
- historical forecasted EPS relying on terms

---
Before delving into the data, let's define what above terms:

**EPS**

> EPS stands for ***Estimated Price Earnings.*** The formal definition of EPS given by Investopedia is this:

Earnings per share is the portion of a company's profit that is allocated to each outstanding share of a common stock, serving as an indicator of the company's financial health.

In other words, the EPS is a portion of the company's **net income** after all of their dividends are paid off. Dividends are profits that are paid out to shareholders of the company. EPS is one of the most useful and valuable financial measurements because they ***determine a stock's worth.*** The higher the stock, the more the company can pay out dividends to its shareholders, and the more net profit they are determined to generate.

$$ EPS = \frac{Net Income - Preferred Dividends}{Weighted Average Common Shares Outstanding}\$$

**EOD**

> EOD stands for the ***End of Day*** price. For any given day, the EOD marks the ***price at which the stock was valued*** at the end of the day's trading period.

In [124]:
PATH = './data/'
PATH_CLEAN = './data/clean/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

### C) Importing the Data <a id = "import"></a>

### Let's summarize the contents of the following DataFrames as we import them.
> All DataFrames consist of the 505 firms found in the 2019 S&P Index with EPS and EOD data encompassing 20 years: from January 1999 until the December 2019.

**Historic forecasted EPS**
> According to Investopedia, consensus estimates is normally an average or median of all the forecasts from individual analysts tracking a particular stock. In this case, the consensus estimate is for ***EPS for each firm present in the index as of 2019.*** Forecasted EPS is calculated by ***quarterly earnings,*** usually by each firm's fiscal period. Estimates of quarterly earnings are published at the beginning of each quarterly period.

In [2]:
#historic forecasted EPS 
df_eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

In [3]:
df_eps_fc.head()

,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,1999Q1,NaN,NaN,NaN,0.025,NaN,NaN,-0.10,0.423,NaN,...,0.456,0.090,0.239,0.123,NaN,NaN,0.123,NaN,0.622,NaN
1,1999Q2,NaN,NaN,NaN,0.020,NaN,NaN,-0.13,0.420,NaN,...,0.213,0.090,0.264,0.135,NaN,NaN,0.150,NaN,0.661,NaN
2,1999Q3,NaN,NaN,NaN,0.023,NaN,NaN,-0.11,0.381,NaN,...,0.767,0.099,0.300,0.129,NaN,NaN,0.171,NaN,0.693,NaN
3,1999Q4,NaN,NaN,NaN,0.016,NaN,NaN,NaN,0.432,NaN,...,0.432,0.124,0.386,0.178,NaN,NaN,0.164,NaN,0.667,NaN
4,00Q1,NaN,NaN,NaN,0.032,NaN,NaN,NaN,0.441,NaN,...,0.283,0.143,0.436,0.138,NaN,NaN,0.138,NaN,0.668,NaN


**Historic actual EPS**
> Unlike forecasted EPS, actual EPS are the real numbers denoting Earnings-per-Share for a singular firm. Historic actual EPS will be compared to forecasted EPS to draw correlations and comparisons.

In [4]:
#historic actual EPS
df_eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

In [5]:
df_eps_act.head()

,Quarter,Year,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,Q1,1999,0.16,0.99,NaN,0.040000,NaN,0.0925,-0.030,0.44,...,0.34,0.0000,0.210,0.123333,1.64,NaN,0.1725,NaN,0.56,NaN
1,Q2,1999,0.35,1.76,NaN,0.035357,NaN,0.1000,-0.145,0.42,...,0.06,0.0975,0.285,0.133333,2.28,NaN,0.2900,NaN,0.60,NaN
2,Q3,1999,0.30,1.86,NaN,0.050357,NaN,0.1075,-0.135,0.30,...,0.63,0.0325,0.315,0.130000,1.96,NaN,0.3200,NaN,0.64,NaN
3,Q4,1999,0.32,1.89,NaN,0.024643,NaN,0.0375,-0.080,0.43,...,0.43,0.1300,0.600,0.180000,1.72,NaN,0.2375,NaN,0.49,NaN
4,Q1,2000,0.29,0.89,NaN,0.040585,NaN,0.1050,-0.045,0.45,...,0.45,0.1650,0.500,0.140000,-1.48,NaN,0.2025,NaN,-0.33,NaN


**Historic actual EOD**
> Though this is not directly related to EPS data, EOD would be an interesting measure to use when generating intriguing visualizations and analyses. Who knows what visuals and conclusions I would arrive to with this measure. 

In [6]:
#historic actual EOD
df_eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

In [7]:
df_eod_act.head()

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,3/31/1999,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,NaN,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,6/30/1999,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,NaN,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,9/30/1999,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,NaN,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,12/31/1999,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,NaN,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,3/31/2000,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,NaN,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


**Historic forecasted EPS 3 months prior**
> Instead of using forecast data collected at the beginning of the fiscal period, this feature contains EPS data projected 3 months before the current fiscal period. This is an interesting metric to see how differently forecasters make their predictions at different times.

In [8]:
#historic forecasted EPS 3-months prior
df_eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

In [9]:
df_eps_fc_terms.head()

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,10/1/1999,00Q1,NaN,NaN,NaN,0.030,NaN,NaN,NaN,0.466,...,NaN,0.143,0.398,0.140,NaN,NaN,0.144,NaN,0.777,NaN
1,1/1/2000,00Q2,NaN,NaN,NaN,0.026,NaN,NaN,NaN,0.437,...,NaN,0.168,0.378,0.153,NaN,NaN,0.184,NaN,0.790,NaN
2,4/1/2000,00Q3,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.414,...,0.84,0.189,0.425,0.149,NaN,NaN,0.218,NaN,0.803,NaN
3,7/1/2000,00Q4,NaN,NaN,NaN,0.032,NaN,NaN,-0.18,0.477,...,0.53,0.211,0.487,0.200,NaN,NaN,0.244,NaN,0.777,NaN
4,9/1/2000,01Q1,NaN,NaN,NaN,0.041,NaN,NaN,-0.09,0.495,...,0.40,0.259,0.475,0.158,NaN,NaN,0.178,NaN,0.750,NaN


## B) Assess <a id = "assess"></a>

> The following DataFrames contain data for each firm across various dates. To account for all firm averages, my goal is to generate a CSV file where each row contains the firm average, with the features as columns.

In [10]:
dict_dfs = {'eps_fcast' : df_eps_fc,
           'eps_actual' : df_eps_act,
           'eod_actual' : df_eod_act,
           'eps_fcast_terms' : df_eps_fc_terms}

### Check Missing Data

In [11]:
for key, df in dict_dfs.items():
    print(key, df.shape)

eps_fcast (84, 506)
eps_actual (84, 507)
eod_actual (84, 506)
eps_fcast_terms (80, 507)


***We need to make sure the number of firms in each DataFrame is consistent.***

---

**Observation 1:** for `eps_fcast`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 506 columns: 1 column being `Term Forecast`, the rest firm names.

**Observation 2:** for `eps_actual`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 507 columns: 2 columns being `Quarter` and `Year`, the rest firm names.

**Observation 3:** for `eod_actual`, there are 505 firms encompassing 84 quarterly calendar periods since 1999.

> There are 506 columns: 1 column being `date`, the rest firm names.

**Observation 4:** for `eps_fcast_terms`, there are 505 firms encompassing 80 quarterly calendar periods since 1999.

> There are 507 columns: 2 columns being `Forecast Made` and `Term Forecasted`, the rest firm names.
 
Since there are only 80 quarterly calendar periods, that ***implies an entire year is missing.***

**Observation 5:** For `eps_fcast`, `eps_actual`, and `eod_actual`, since there are 4 quarters in a year, 84 quarterly forecast periods equate to 21 years. This is correct since we are analyzing the years from 1999 until the end of 2019.

### Most importantly, the number of firms across all DataFrames is consistent.

----

***Check which year is missing from `eps_fcast_terms`***.

> Each **Term Forecasted** entry under `eps_fcast_terms` records the year with 2 digits, so Quarter 1 of the year 2000 becomes 00Q1.
- isolate the first 2 characters to get the year
- join a '20' in front of the string so 00 becomes 2000
- list the number of unique values.

In [12]:
#iterate over all years in eps_fcast_terms, append '20' in front of string
list(map(lambda x: '20' + x, df_eps_fc_terms['Term Forecasted'].str[:2].unique()))

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

**Observation**: The year 1999 is missing from `df_eps_fc_terms`. This makes sense because the start of the forecasting period would be in the last quarter of 1999, which is October. 

---

### Check Nulls

In [13]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {}: {}'.format(key, df.isna().sum().sum()))

# NaN in eps_fcast: 7055
# NaN in eps_actual: 5021
# NaN in eod_actual: 6921
# NaN in eps_fcast_terms: 7080


**Observation:** All four DataSets contain null values.
> In order to combat this, we'll have to look at both the **number of rows** and **number of columns** with missing data, separately. This way, we can isolate which firms and/or time periods contain complete or incomplete data.

In [14]:
#check rows for missing data
for key, df in dict_dfs.items():
    num_rows_missing = df.isna().sum().max()
    print('{} has {} time periods containing missing data, out of {} total rows.'.format(key, num_rows_missing, df.shape[0]))

eps_fcast has 82 time periods containing missing data, out of 84 total rows.
eps_actual has 84 time periods containing missing data, out of 84 total rows.
eod_actual has 83 time periods containing missing data, out of 84 total rows.
eps_fcast_terms has 80 time periods containing missing data, out of 80 total rows.


In [15]:
len(df_eps_fc.columns[df_eps_fc.isnull().any()])

254

In [16]:
#check columns for missing data
for key, df in dict_dfs.items():
    cols_missing = df.columns[df.isnull().any()]
    num_cols_missing = len(cols_missing)
    print('{} has {} firms containing missing data, out of {} total columns.'
         .format(key, num_cols_missing, df.shape[1]))

eps_fcast has 254 firms containing missing data, out of 506 total columns.
eps_actual has 432 firms containing missing data, out of 507 total columns.
eod_actual has 505 firms containing missing data, out of 506 total columns.
eps_fcast_terms has 340 firms containing missing data, out of 507 total columns.


**Observation 1:** The only two datasets with incomplete data for all time periods are **actual EPS** and **forecasted EPS 3 months prior.**

> To address this problem, it'd be helpful to isolate the time period ranges for the datasets with incomplete data by row, **forecasted EPS** and **actual EOD price.**

**Observation 2:** For all datasets, all firms contain incomplete data across all time periods.
> This is expected, as analyzing financial history spanning over 20 years will naturally be rife with missing and inaccurate data. The ***good news is that `eps_fcast`, `eps_actual`, and `eps_fcast_terms` are the most complete, while `eod_actual` contains the most amount of missing data.***


**Moving forward, we need to make sure that these inconsistencies won't clash with our analysis.**
> **My approach:** instead of looking at rows and columns ***with*** missing data, we'll be looking at rows and columns that ***are all missing data.***

I figured that if there is some missing data here and there scattered throughout the matrix, then that should not skew our analysis too much.

However, if there a significant amount of rows/columns that are entirely empty, then we ***might have to get ready to drop some dates and firms from our data overall.***

In [17]:
#check for empty rows, return False if row contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_rows = (df[cols_check].isnull().apply(lambda x: all(x), axis = 1)).value_counts()
    print(key, '\n', num_empty_rows, '\n----')

eps_fcast 
 False    84
dtype: int64 
----
eps_actual 
 False    84
dtype: int64 
----
eod_actual 
 False    84
dtype: int64 
----
eps_fcast_terms 
 False    80
dtype: int64 
----


**Observation:** All datasets do not contain empty rows.
> This is good news, since we can rely on the firms' averages per row instead of having to drop or limit time periods.

In [18]:
#check for empty columns, return False if column contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_cols = df[cols_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()
    print(key, '\n', num_empty_cols)

eps_fcast 
 False    506
dtype: int64
eps_actual 
 False    506
True       1
dtype: int64
eod_actual 
 False    506
dtype: int64
eps_fcast_terms 
 False    506
True       1
dtype: int64


**Observation:** `eps_actual` and `eps_fcast_terms` are the only datasets that have an empty column.
> Let's isolate and look at the singular empty column for both DataFrames.

In [19]:
#create function to return an array of column names containing empty data
def comb_cols(df):
    empty_cols = []
    for column in df:
        if df[column].isnull().all():
            empty_cols.append(column)
            
    return empty_cols

In [20]:
#comb datasets for empty columns
print('In eps_act, the firm {} has no data.'.format(comb_cols(df_eps_act)))
print('In eps_fc_terms, the firm {} has no data.'.format(comb_cols(df_eps_fc_terms)))

In eps_act, the firm ['AMCR UN Equity'] has no data.
In eps_fc_terms, the firm ['AMCR UN Equity'] has no data.


**Observation:** The same firm in both datasets is empty of data.
> Though this is an annoying error to deal with, it still is to our advantage that both datasets ***share one firm*** in common for missing data. This way, we don't have to worry about dropping two entire firms.

---

### Check Duplicate Data

In [21]:
#check for duplicate data across all rows and columns
for key, df in dict_dfs.items():
    print(key, df.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate data.*** This is good news!

**Next, I will check for duplicated firm names.** Although the presence of duplicated firm names will inherently imply duplicated data, sometimes data gets dispersed in weird, unexpected ways, especially when dealing with large datasets.

In [22]:
#check for duplicated firm names
for key, df in dict_dfs.items():
    print(key, df.columns.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate firm names.*** This is also good news.

### There is no need to dedupe our data during the cleaning stage.

---

### Check Data Types

> Make sure all numerical data types are consistent.
- EPS and EOD values must all be **floats**
- Dates should all be **objects** (for now)

- `df_eps_fc` contains ***1 date field,***, so we should expect ***1 object type.***
- `df_eps_act` contains ***2 date fields,*** so we should expect ***2 object types.***
- `df_eod_act` contains ***1 date field,*** so we should expect ***1 object type.***
- `df_eps_fc_terms` contains ***2 date fields,*** so we should expect ***2 object types.***

> And everything else should be a ***float*** type.

**Examine each dataset's overall data types per column.**

In [23]:
for key, df in dict_dfs.items():
    print(key,'\n', df.dtypes.value_counts(), '\n--------')

eps_fcast 
 float64    505
object       1
dtype: int64 
--------
eps_actual 
 float64    499
object       7
int64        1
dtype: int64 
--------
eod_actual 
 float64    505
object       1
dtype: int64 
--------
eps_fcast_terms 
 float64    505
object       2
dtype: int64 
--------


**Observation 1:** `df_eps_fc`, `df_eod_act`, and `df_eps_fc_terms` are all consistent with the expected number of object types.

> But we'll still have to double check these facts later.

**Observation 2:** `df_eps_act` contains 5 more object columns than expected.

---

***Isolate 'Object' Columns under `df_eps_act`***

In [24]:
df_eps_act.select_dtypes(include = 'object').head()

,Quarter,BRK/B UN Equity,FOX UW Equity,GOOG UW Equity,HCP UN Equity,SYMC UW Equity,UA UN Equity
0,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1,Q2,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
2,Q3,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
3,Q4,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


**Observation:** These firms are all 'Object' types because there is no recorded data under them.
> This is actually missing data. From `df_eps_act`, there are 6 firms that are empty of data.

### This was a tricky situation to spot, but I caught it in the end. There actually is missing data after all, but we missed those 5 entire firms during the "Check Missing Data" stage, because they were recorded as objects. This means that Pandas wrongly recognized these 5 firms as "complete."

### Quality

**Missing Data**
> **Deal with dropping/keeping missing data in the end when all data is properly cleaned.**

-  `df_eps_fc_terms` is missing the year 1999.
- `df_eps_act` and `df_eps_fc_act` have one firm with empty data: 'AMCR UN Equity'
--- 
- Unnormalized date formats among EPS and EOD data.
- Firm names across all DataFrames are capitalized and contain white space.
- Erroneous data type for `df_eps_act` Object columns.
- No recorded 20-year for each dataset.
- No recorded yearly averages for each dataset
- No recorded quarterly averages for each dataset.

### Tidiness
- Firm 20-year, yearly, and quarterly averages contained in different DataFrames.
- forecast + actual EPS DataFrames need to be merged with firm names transposed into rows


## C) CLEAN



In [233]:
eps_fc_clean = df_eps_fc.copy()
eps_act_clean = df_eps_act.copy()
eod_act_clean = df_eod_act.copy()
eps_fc_terms_clean = df_eps_fc_terms.copy()

In [234]:
#create dictionary of all clean dfs for iteration
dict_clean = {'eps_fc_clean' : eps_fc_clean,
             'eps_act_clean' : eps_act_clean,
             'eod_act_clean' : eod_act_clean,
             'eps_fc_terms_clean' : eps_fc_terms_clean}

### Code
> **ISSUE 1:** Unnormalized date formats among EPS and EOD data.

**Define:** 
- Conjoin `eps_act_clean` dates from 2 columns into 1 to match `eps_fc_clean` format.
> The format we want is YYYYQN. For example, Quarter 1 in 2005 will be 2005Q1.


- Rename both fiscal periods to **term_forecast.**

- For both fiscal periods under `eps_fc_terms_clean`, add an underscore.

In [235]:
#create new column, combine year and quarter into 1 string
eps_act_clean['term_forecast'] = eps_act_clean['Year'].map(str) + eps_act_clean['Quarter']

In [236]:
#drop 'Year' and 'Quarter'
eps_act_clean.drop(['Year', 'Quarter'], axis = 1, inplace = True)

In [237]:
#designate new column as first column
reorder_cols = eps_act_clean.columns.tolist()
reorder_cols.insert(0, reorder_cols.pop(reorder_cols.index('term_forecast')))

eps_act_clean = eps_act_clean.reindex(columns = reorder_cols)

In [238]:
#rename eps_fc_clean fiscal period to term_forecast
eps_fc_clean.rename(index = str, columns = {'Term Forecasted' : 'term_forecast'}, inplace = True)

#reassign to dictionary
dict_clean['eps_act_clean'] = eps_act_clean

In [239]:
eps_fc_clean[4:].term_forecast = '20' + eps_fc_clean[4:].term_forecast.astype(str)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [240]:
#add an underscore for both date periods under eps_fc_terms_clean
eps_fc_terms_clean.rename(index = str, columns = {'Forecast Made' : 'forecast_made',
                                                 'Term Forecasted' : 'term_forecasted'},
                         inplace = True)

### Test

In [241]:
#give overview of both DataFrames
eps_fc_clean.sample(3)

,term_forecast,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
72,2017Q1,0.490,0.572,2.155,3.218,1.262,1.230,0.231,0.430,1.496,...,0.486,0.559,0.860,0.501,0.633,0.38,0.592,2.111,0.539,0.481
5,2000Q2,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.438,NaN,...,0.250,0.168,0.519,0.152,NaN,NaN,0.177,NaN,0.690,NaN
27,2005Q4,0.363,-2.360,0.361,0.053,NaN,0.239,0.000,0.758,0.359,...,0.256,0.202,1.451,0.350,1.280,NaN,0.390,0.827,1.348,NaN


In [242]:
eps_act_clean.sample(3)

,term_forecast,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
57,2013Q2,0.48,0.88,1.60,1.451429,0.67,0.200,0.14,0.30,1.70,...,0.40,0.47,1.55,0.61,0.88,0.25,0.62,0.90,0.30,0.26
41,2009Q2,-0.29,-1.39,0.84,0.260000,NaN,0.475,-0.18,0.83,0.66,...,0.25,0.29,0.82,0.47,0.64,NaN,0.65,0.98,-0.21,NaN
40,2009Q1,0.18,-1.35,0.99,0.362857,NaN,0.360,-0.28,0.93,0.77,...,0.38,0.30,0.92,0.42,0.20,NaN,0.47,0.91,-7.47,NaN


In [243]:
#check consistent format between both `term_forecast` fields
forecasts = pd.concat([eps_fc_clean.term_forecast, eps_act_clean.term_forecast])

#count number of instances where unique term_forecast values are the same
(forecasts.value_counts().sort_index() == 2).value_counts()

True    84
Name: term_forecast, dtype: int64

In [244]:
#check that dates are the first columns
for key, df in dict_clean.items():
    print(df.columns[:2])

Index(['term_forecast', 'A UN Equity'], dtype='object')
Index(['term_forecast', 'A UN Equity'], dtype='object')
Index(['date', 'A UN Equity'], dtype='object')
Index(['forecast_made', 'term_forecasted'], dtype='object')


### Code
> **ISSUE 2:** Firm names across all DataFrames are capitalized and contain white space.

**Define:** Iterate across all DataFrames. Get rid of everything from the first whitespace character onward. Lowercase column names.

In [245]:
#lowercase names
for key, df in dict_clean.items():
    lower_cols = [x.lower() for x in df.columns]
    df.columns = lower_cols

In [246]:
#get rid of white space
for key, df in dict_clean.items():
    stock_symbols = [x.split(' ', 1)[0] for x in df.columns]
    df.columns = stock_symbols

### Test

In [247]:
#check that all firm names are consistent, should return 505 firm results
all_ticks = []
for key, df in dict_clean.items():
    firm_ticks = np.array(df.columns.values)
    all_ticks.append(firm_ticks)
    
#flatten array
all_ticks = np.hstack(all_ticks)

#assign tick names to Pandas Series
all_ticks = pd.Series(all_ticks)

In [248]:
#look at lowest 3 value counts, SHOULD BE DATE PERIODS
all_ticks.value_counts().sort_values()[:5]

date               1
term_forecasted    1
forecast_made      1
term_forecast      2
es                 4
dtype: int64

In [249]:
#look at highest value counts, should be FOUR (because there are 4 dataframes all with firm names)
all_ticks.value_counts().sort_values(ascending = False)[:5]

tfx     4
wltw    4
aiv     4
prgo    4
zts     4
dtype: int64

In [250]:
#check for any upper case instance among columns
all_ticks.str.isupper().any()

False

In [251]:
#check for any white spae
all_ticks.str.contains(' ').any()

False

### Code
> **Issue 3:** Erroneous data type for `df_eps_act` Object columns.

**Define:**

- Convert '#N/A Field Not Applicable' strings into 'NaN'
- Convert the 6 `eps_act_clean` Object columns into NaN type.

In [255]:
#isolate 6 firm names recorded as 'object'
obj_ticks = eps_act_clean.select_dtypes(include = 'object').columns.values

#exclude 'term_forecast'
obj_ticks = obj_ticks[1:]

In [256]:
#convert to NaN
str_replace = '#N/A Field Not Applicable'
eps_act_clean[obj_ticks] = eps_act_clean[obj_ticks].replace(str_replace, np.nan)

### Test

In [257]:
eps_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, term_forecast to zts
dtypes: float64(505), object(1)
memory usage: 332.1+ KB


In [258]:
#make sure 'term_forecast' is the only object-type column
eps_act_clean.select_dtypes(include = 'object').columns.values.tolist()

['term_forecast']

### Code
> **Issue 4:** No recorded 20-year averages for each dataset.

**Define:**
- Isolate 20-year averages for each firm into its own DataFrame

- Create new DataFrame `df_twenty_avgs` depicting all 20-year averages for each firm

In [259]:
#grab all firm names
firm_names = eps_act_clean.columns.values[1:]

In [260]:
def get_twenty_yr_avg(df, index_names, col_name):
    return pd.DataFrame({col_name : df[index_names].mean()},
                       index = index_names)

In [261]:
#get 20-year averages
eps_fc_twenty_avg = get_twenty_yr_avg(eps_fc_clean, firm_names, 'eps_fc')
eps_act_twenty_avg = get_twenty_yr_avg(eps_act_clean, firm_names, 'eps_act')
eod_act_twenty_avg = get_twenty_yr_avg(eod_act_clean, firm_names, 'eod_act')
eps_fc_terms_twenty_avg = get_twenty_yr_avg(eps_fc_terms_clean, firm_names, 'eps_fc_terms')

In [262]:
#put averages into a list
twenty_avg_list = [eps_fc_twenty_avg,
                  eps_act_twenty_avg,
                  eod_act_twenty_avg,
                  eps_fc_terms_twenty_avg]

In [263]:
#merge all Series
df_twenty_year_avgs = pd.concat(twenty_avg_list, axis = 1, sort=False)

### Test

In [264]:
df_twenty_avgs.sample(10)

,eps_fc,eps_act,eod_act,eps_fc_terms
fcx,0.413905,0.023542,22.804506,0.583325
tgt,0.776250,0.744167,53.807622,0.802850
cost,0.845768,0.876102,89.863205,0.876112
ko,0.367774,0.324880,32.090454,0.392900
tif,0.579723,0.579286,56.222124,0.609212
hban,0.247310,0.159044,14.205866,0.276162
rcl,0.806345,0.820843,48.978825,0.946679
expd,0.364381,0.374639,37.380280,0.390610
bxp,0.587579,0.853735,85.165410,0.535673
emn,0.918964,0.834854,45.070348,1.058143


In [265]:
#check missing data
df_twenty_year_avgs.isna().any()

eps_fc          False
eps_act          True
eod_act         False
eps_fc_terms     True
dtype: bool

In [266]:
#check duplicate data
df_twenty_avgs.duplicated().any()

False

### Code
> **Issue 5:** No recorded yearly averages for each dataset.

**Define:**
- Isolate yearly averages for each firm into its own DataFrame
- Create new DataFrame `df_yearly_avgs` depicting all yearly averages for each firm

In [267]:
#separate DataFrames by year
bin_years = ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
             '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
             '2019']

In [268]:
eps_fc_bins = ['eps_fc_' + s for s in bin_years]

In [280]:
eps_fc_clean.groupby(eps_fc_clean.term_forecast.str[:4]).mean()

,a,aal,aap,aapl,abbv,abc,abmd,abt,acn,adbe,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
term_forecast,,,,,,,,,,,,,,,,,,,,,
1999,NaN,NaN,NaN,0.02100,NaN,NaN,-0.113333,0.41400,NaN,0.08575,...,0.46700,0.10075,0.29725,0.14125,NaN,NaN,0.15200,NaN,0.66075,NaN
2000,NaN,NaN,NaN,0.02850,NaN,NaN,-0.180000,0.44425,NaN,0.12550,...,0.43150,0.17775,0.54325,0.15900,NaN,NaN,0.17350,NaN,0.71175,NaN
2001,NaN,NaN,0.02850,-0.00575,NaN,NaN,-0.128750,0.47050,0.18250,0.13200,...,0.56225,0.27925,0.55475,0.16800,NaN,NaN,0.19700,0.24100,0.79950,NaN
2002,NaN,NaN,0.21075,0.00575,NaN,NaN,-0.292000,0.51825,0.22100,0.10900,...,0.40300,0.05850,0.43950,0.22200,NaN,NaN,0.22975,0.29675,0.91600,NaN
2003,NaN,NaN,0.31850,0.00250,NaN,NaN,-0.315000,0.55175,0.24750,0.13025,...,0.30025,0.11850,0.59875,0.25575,NaN,NaN,0.24775,0.41475,1.00775,NaN
2004,NaN,NaN,0.41400,0.01000,NaN,NaN,-0.240000,0.57450,0.29900,0.20325,...,0.29200,0.18175,0.88000,0.28625,NaN,NaN,0.28600,0.55575,1.11200,NaN
2005,0.277667,-1.58000,0.52000,0.04150,NaN,0.22800,-0.013333,0.62450,0.35450,0.26775,...,0.30600,0.22625,1.31050,0.32425,1.00000,NaN,0.32625,0.73150,1.27650,NaN
2006,0.395250,0.20700,0.54375,0.07175,NaN,0.25225,-0.158000,0.61975,0.38950,0.29225,...,0.30625,0.24175,1.53325,0.34425,1.10750,NaN,0.34925,0.85225,1.39200,NaN
2007,0.423750,0.38225,0.58150,0.10675,NaN,0.31325,-0.211750,0.69825,0.46500,0.38050,...,0.37075,0.26225,1.80925,0.40225,0.98000,NaN,0.38700,0.96700,1.20300,NaN


***CHECK THAT OUR DATA IS CONSISTENT WITH THE ORIGINAL DATA***

***REORDER COLUMNS (IF APPLICABLE)***

# III) Store Data

In [132]:
df_twenty_year_avgs.to_csv(PATH_CLEAN + 'twenty_year_avgs.csv', encoding = 'utf-8', index = False)

# IV) Notes

- I did not normalize between (`eod_act_` and `eps_fc_terms`) and (`eps_fc` and `eps_act`) because the former two are recorded on calendar dates, while the latter two are recorded on fiscal periods.
- When dealing with the CSVs depicting averages, we need to keep in mind **missing data** in our interpretations.


In [ ]:
#convert notebook to HTML